In [18]:
import requests as req
import json as js
from dtos.dtos import *

In [19]:
baseUrl:str = "http://localhost:3010/maas"
baseHeader = {"user" : "{'_id' : '60f527b78c90cf6b8bf450eb', 'client_id' : '60f1b8e0203c9b20acb9ad26'}", 
              "Content-Type" : "application/json"}
main_date = "2024-05-28T15:00:36.459Z"
provider_id = "0b13f2d6-d5cb-48aa-aeed-ed3a5a7f1da6"

<h1> Check the auth of Cooltra <h1>

In [3]:
dataAuth = """
    {
     "grant_type": "client_credentials",
     "client_id": "JTrLV2ByJYJ5w1FU0M8tjkrnOh2U8uD",
     "client_secret": "zEoTATE6iciep+t8iHfbLmmTbJe3k54C2ZpCavXTXe/ZO0FJqbhUsM0prGv/JwgQ",
     "scope": "offers"
    }
           """

# authorize
#########################################################################
cooltraBaseUrl = "https://maas.zeus-stage.cooltra.com"

response = req.request("POST",
                       url=cooltraBaseUrl + "/oauth/token",
                       headers= {"Content-Type" : "application/json", "Accept" : "application/json"},
                       data=dataAuth
                       )

result : CooltraAuthResponseDTO = CooltraAuthResponseDTOSchema().loads(response.text)
token = "Bearer " + result.access_token

#########################################################################


In [4]:
token

'Bearer eyJhbGciOiJSUzI1NiIsImtpZCI6ImRmOGIxNTFiY2Q5MGQ1YjMwMjBlNTNhMzYyZTRiMzA3NTYzMzdhNjEiLCJ0eXAiOiJKV1QifQ.eyJwcm92aWRlcl9pZCI6Ik1PVkVTSU9OIiwiaXNzIjoiaHR0cHM6Ly9zZWN1cmV0b2tlbi5nb29nbGUuY29tL2Nvb2x0cmEtc3RhZ2UiLCJhdWQiOiJjb29sdHJhLXN0YWdlIiwiYXV0aF90aW1lIjoxNzE4NzE4NzU4LCJ1c2VyX2lkIjoiZXJMVlVwN21IcGI3S1NLdXk3d1hBaG93NU1iMiIsInN1YiI6ImVyTFZVcDdtSHBiN0tTS3V5N3dYQWhvdzVNYjIiLCJpYXQiOjE3MTg3MTg3NTgsImV4cCI6MTcxODcyMjM1OCwiZW1haWwiOiJqdHJsdjJieWp5ajV3MWZ1MG04dGprcm5vaDJ1OHVkQGNvb2x0cmEuY29tIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJmaXJlYmFzZSI6eyJpZGVudGl0aWVzIjp7ImVtYWlsIjpbImp0cmx2MmJ5anlqNXcxZnUwbTh0amtybm9oMnU4dWRAY29vbHRyYS5jb20iXX0sInNpZ25faW5fcHJvdmlkZXIiOiJwYXNzd29yZCJ9fQ.NV_sB609PQ_-CiRbJ3RgS-SJ9BR2jtRlRStWYvmmRdX23Dop_Pc0oJHRVwfXyp7kyAsLDDcqJXZjI24MRLzSB_dPkDgCxX0HBfaS0pTyfllxcg88zACkuuYChfwZp_ViHXK4Z7TFzA22E42aKtJJpa2s6gH8F-Sri1MUjuQt3dAWjWD0UAHMQxueVOYnzZCFE0tVvv323NZhliuI0Gs_HaiohaL13X8q8uq6TVuF5nQVlRcN3aiGI2qmzvCSC48JfYhvOnoetY0djbapRChAeNR3S26KqaWWPgMHQMPqYSF3o_7drHx7EOBwanTx0z

<h1> User registration </h1>

In [20]:
newUser = CooltraUserRegistrationRequestDTO(
    external_user_id="f468fdd1-c6f3-4dca-9dc3-dbcba6b8ea8c",
    first_name="John",
    middle_name="Doe",
    last_name="Doe",
    email="test@test.com",
    phone="+393493494949",
    language="en",
    gender="m",
    birthdate="1990-01-01",
    address_street="123 Main St",
    address_city="New York",
    address_zip="12345",
    address_country_code="US",
    address_state="NY",
    provider="erLVUp7mHpb7KSKuy7wXAhow5Mb2",
    provider_code="cooltra"
)


response = req.request("POST", 
            baseUrl + "/moto-rental/register",
            headers=baseHeader,
            data=CooltraUserRegistrationRequestDTOSchema().dumps(newUser)
            )



In [21]:
response.text

'{"external_id":"136382391535434","first_name":"John","middle_name":null,"last_name":"Doe","gender":null,"phone":null,"language":null,"birthdate":"0001-01-01T00:00:00","address_street":null,"address_city":null,"address_zip":null,"address_state":null,"address_country_code":null,"home_system_id":null,"user_condition":null,"email":"test@test.com","state":null}'

<h1> Get configuration </h1>

In [5]:
response = req.request("GET", 
                       baseUrl + "/moto-rental/configuration", 
                       headers=baseHeader,
                       params={"providerCode":"cooltra"})
result : CooltraConfigurationResponseDTO = CooltraConfigurationResponseDTOSchema().loads(response.text)
all_ids = [system.id for system in result.systems]

In [6]:
all_ids

['rome',
 'paris',
 'madrid',
 'berlin',
 'zurich',
 'valencia',
 'seville',
 'turin',
 'lisbon',
 'barcelona',
 'milan']

<h1> Get vehicles in specific zone </h1>

In [57]:
response = req.request("GET", 
                       baseUrl + "/moto-rental/vehicles", 
                       headers=baseHeader, 
                       params={"providerCode":"cooltra", "systemId" : "barcelona"})
vehicles:List[CooltraMotoRentalVehicleDTO] = CooltraMotoRentalVehicleDTOSchema().loads(response.text, many=True)
first_vehicle_id = vehicles[5].id

<h1> Get specific vehicle </h1>

In [58]:
response = req.request("GET", 
                       baseUrl + f"/moto-rental/vehicles/{first_vehicle_id}", 
                       headers=baseHeader, 
                       params={"providerCode":"cooltra", "systemId" : "rome"})
result : CooltraMotoRentalVehicleDTO = CooltraMotoRentalVehicleDTOSchema().loads(response.text)

In [59]:
result

CooltraMotoRentalVehicleDTO(id=17990347497, license_plate='FAKE-L2768CYJ', position=[2.168419217412566, 41.40277256088423], range=50, battery_percentage=25, model_id=None)

<h1> Reserve vehicle </h1>

In [61]:
reservationBody = CooltraReservationRequestDTO(
    vehicle_id = first_vehicle_id,
    provider_code="cooltra"
)

response = req.request("POST", 
                       baseUrl + f"/moto-rental/reserve", 
                       headers=baseHeader, 
                       data=CooltraReservationRequestDTOSchema().dumps(reservationBody))

result : CooltraReservationInfoDTOSchema = CooltraReservationInfoDTOSchema().loads(response.text)

ValidationError: {'id': ['Missing data for required field.'], 'system_id': ['Missing data for required field.'], 'created_at': ['Missing data for required field.'], 'expires_at': ['Missing data for required field.'], 'expires_in': ['Missing data for required field.'], 'vehicle': ['Missing data for required field.'], 'errors': ['Unknown field.'], 'code': ['Unknown field.']}

'{"errors":["Cooltra Error : {\\r\\n  \\u0022user\\u0022: [\\r\\n    \\u0022max_reservations_reached\\u0022\\r\\n  ]\\r\\n}"],"code":"cooltra__max_reservations_reached"}'

<h1> Check in to vehicle </h1>

In [55]:
reservationBody = CooltraCheckInRequestDTO(
    provider_code = "cooltra",
    vehicle_id = first_vehicle_id
)

response = req.request("POST", 
                       baseUrl + f"/moto-rental/check-in", 
                       headers=baseHeader, 
                       data=CooltraCheckInRequestDTOSchema().dumps(reservationBody))

result : CooltraCheckInInfoDTO = CooltraCheckInInfoDTOSchema().loads(response.text)

ValidationError: {'id': ['Missing data for required field.'], 'system_id': ['Missing data for required field.'], 'reservation_id': ['Missing data for required field.'], 'started_at': ['Missing data for required field.'], 'duration': ['Missing data for required field.'], 'state': ['Missing data for required field.'], 'vehicle': ['Missing data for required field.'], 'errors': ['Unknown field.'], 'code': ['Unknown field.']}

In [56]:
response.text

'{"errors":["Cooltra Error : {\\r\\n  \\u0022vehicle\\u0022: [\\r\\n    \\u0022must be available\\u0022\\r\\n  ]\\r\\n}"],"code":"cooltra__vehicle_is_no_longer_available"}'